# Visión general de las estructuras de datos de Pandas
En esta sección, discutiremos las clases `Series`, `Index`, y `DataFrame`. Para ello, leeremos un fragmento del fichero CSV con el que trabajaremos más adelante. Pero no te preocupes por esa parte todavía.

## Acerca de los datos
En este cuaderno trabajaremos con 5 filas de los datos de terremotos recogidos entre el 18 de septiembre de 2018 y el 13 de octubre de 2018 (obtenidos del US Geological Survey (USGS) mediante la [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/))

## Trabajando con Arrays NumPy
Vamos a leer un archivo CSV corto (usando `numpy`) para algunos datos de muestra.

In [ ]:
import numpy as np

data = np.genfromtxt(
    'data/example_data.csv', delimiter=';', 
    names=True, dtype=None, encoding='UTF'
)
data

Podemos encontrar las dimensiones con el atributo `shape`:

In [ ]:
data.shape

Podemos encontrar los tipos de datos con el atributo `dtype`:

In [ ]:
data.dtype

Cada una de las entradas de la matriz es una fila del archivo CSV. Los arrays de NumPy contienen un único tipo de datos (a diferencia de las listas, que permiten tipos mixtos); esto permite realizar operaciones rápidas y vectorizadas. Cuando leemos los datos, obtenemos un array de objetos `numpy.void`, que se crean para almacenar tipos flexibles. Esto se debe a que NumPy tiene que almacenar varios tipos de datos diferentes por fila: cuatro a
string, un float y un int. Esto significa que no podemos aprovechar las mejoras de rendimiento que NumPy proporciona para los objetos de un solo tipo de datos.

Digamos que queremos encontrar la magnitud máxima&mdash;podemos usar una **[comprensión de lista](https://www.python.org/dev/peps/pep-0202/)** para seleccionar el tercer índice de cada fila, que se representa como un objeto `numpy.void`. Esto hace una lista, lo que significa que podemos tomar el máximo usando la función `max()`:

In [ ]:
%%timeit
max([row[3] for row in data])

Si, en cambio, creamos un array NumPy para cada columna, esta operación es mucho más fácil (y eficiente) de realizar. Podemos usar un **[dictionary comprehension](https://www.python.org/dev/peps/pep-0274/)** para hacer un diccionario donde las claves son los nombres de las columnas y los valores son arrays NumPy de los datos:

In [ ]:
array_dict = {
    col: np.array([row[i] for row in data])
    for i, col in enumerate(data.dtype.names)
}
array_dict

Obtener la máxima magnitud es ahora simplemente cuestión de seleccionar la tecla `mag` y llamar al método `max()`. Esto es casi dos veces más rápido que la implementación de comprensión de lista cuando se trata de sólo 5 entradas, imagina cuánto peor será el primer intento en grandes conjuntos de datos:

In [ ]:
%%timeit
array_dict['mag'].max()

Sin embargo, esta representación presenta otros problemas. Supongamos que queremos obtener toda la información sobre el seísmo de magnitud máxima. Tendríamos que encontrar el índice del máximo y luego, para cada una de las claves del diccionario, tomar ese índice:

In [ ]:
np.array([
    value[array_dict['mag'].argmax()] 
    for key, value in array_dict.items()
])

El resultado es ahora una matriz NumPy de cadenas (nuestros valores numéricos fueron convertidos), y ahora estamos en el formato de antes. Además, considere la posibilidad de tratar de ordenar los datos por magnitud de menor a mayor. En la primera representación, tendríamos que ordenar las filas examinando el 3er índice. Con la segunda representación, tendríamos que determinar el orden de los índices de la columna `mag`, y luego ordenar todas las demás matrices con esos mismos índices. Está claro que trabajar con varias matrices NumPy de diferentes tipos de datos a la vez es un poco engorroso. Sin embargo, `pandas` se construye sobre arrays NumPy para hacer esto más fácil. Comencemos nuestra exploración de `pandas` con una visión general de las estructuras de datos.

## `Series`
La clase `Series` proporciona una estructura de datos para arrays de un solo tipo con algunas funcionalidades adicionales.

In [ ]:
import pandas as pd

place = pd.Series(array_dict['place'], name='place')
place

Estos son algunos de los atributos más utilizados con los objetos `Series`:

|Atributo | Devuelve |
| --- | --- |
| `Name`   | Nombre: El nombre del objeto `Series`.
| `Type`   | Tipo de datos del objeto Serie
| `Shape`  | Dimensiones del objeto `Series` en una tupla de la forma `(número de filas,)`.
| `Index`  | El objeto `Index` que forma parte del objeto `Series`.
| `values` | Los datos del objeto `Series` |

En su mayor parte, los objetos `pandas` utilizan arrays NumPy para sus representaciones internas de datos. Sin embargo, para algunos tipos de datos, `pandas` se basa en NumPy para crear sus propios [arrays](https://pandas.pydata.org/pandas-docs/stable/reference/arrays.html). Por esta razón, dependiendo del tipo de datos, `values` puede ser un objeto `pandas.array` o `numpy.array`. Por lo tanto, si necesitamos asegurarnos de que nos devuelve un tipo específico, entonces se recomienda utilizar el atributo `array` o el método `to_numpy()`, respectivamente, en lugar de `values`.

Veamos ahora algunos ejemplos utilizando estos atributos.

### Obtener el nombre de la serie
El array de NumPy contenía el nombre de los datos en el atributo `dtype`; aquí podemos acceder a él directamente:

In [ ]:
place.name

### Obtención del tipo de dato
Un objeto `Series` contiene un único tipo de datos. Aquí es `'O'` para objeto.

In [ ]:
place.dtype

### Obtener las dimensiones de la serie
Al igual que con NumPy, podemos utilizar `shape` para obtener las dimensiones como `(filas, columnas)`. Los objetos `Series` son de una sola columna, por lo que sólo tienen valores para la dimensión rows.

In [ ]:
place.shape

### Aislar los valores de la serie
Este objeto `Series` está almacenando sus valores como un array NumPy:

In [ ]:
place.values

## `Index`
La adición de la clase `Index` hace que la clase `Series` sea más potente que un array NumPy. Podemos obtener el índice del atributo `index` de un objeto `Series`:

In [ ]:
place_index = place.index
place_index

Al igual que con los objetos `Series`, podemos acceder a los datos subyacentes a través del atributo `values`. Ten en cuenta que este objeto `Index` también se construye sobre un array NumPy:

In [ ]:
place_index.values

Estos son algunos de los atributos más utilizados con los objetos `Index`:

|Atributo | Return|
| --- | --- |
| `name` | El nombre del objeto `Index |
| `dtype` | Tipo de datos del objeto `Index |
| `shape` | Dimensiones del objeto `Index` |
| `values` | Los datos del objeto `Index` |
| `is_unique` | Comprueba si el objeto `Index` tiene todos los valores únicos |

Podemos comprobar el tipo de los datos subyacentes, igual que con un objeto `Series`:

In [ ]:
place_index.dtype

Lo mismo para las dimensiones:

In [ ]:
place_index.shape

Podemos comprobar si los valores son únicos:

In [ ]:
place_index.is_unique

Con NumPy podemos realizar operaciones aritméticas elemento a elemento entre matrices:

In [ ]:
np.array([1, 1, 1]) + np.array([-1, 0, 1])

Pandas también lo soporta, y el índice determina cómo se realizan las operaciones elemento a elemento. Con la suma, sólo se suman los índices coincidentes:

In [ ]:
numbers = np.linspace(0, 10, num=5) # makes numpy array([0, 2.5, 5, 7.5, 10])
x = pd.Series(numbers) # index is [0, 1, 2, 3, 4]
y = pd.Series(numbers, index=pd.Index([1, 2, 3, 4, 5]))
x + y

No estamos limitados a los índices enteros de las estructuras tipo lista, y podemos etiquetar nuestras filas. Las etiquetas pueden modificarse en cualquier momento y ser cosas como fechas o incluso otra columna. En el capítulo 3, veremos cómo realizar algunas operaciones sobre el índice para modificarlo. Después, en el capítulo 4, utilizaremos el índice para operaciones de fusión de datos y agregación de los mismos.

## `DataFrame`
Tener un objeto `Series` para cada columna es una mejora respecto a la representación de NumPy; sin embargo, seguimos teniendo el mismo problema cuando queremos ordenar basándonos en un valor o sacar una fila entera. El `DataFrame` nos da una representación de una tabla formada por muchos objetos `Series` que forman las columnas y un objeto `Index` compartido que etiqueta las filas. Podemos crear un objeto `DataFrame` a partir de cualquiera de las representaciones de NumPy con las que trabajábamos antes (también podríamos crear un objeto `Series` para cada columna, pero no es necesario hacerlo):

In [ ]:
df = pd.DataFrame(array_dict) 

# esto también funcionará con la primera representación
# df = pd.DataFrame(data)

df

Podemos comprobar el tipo de los datos subyacentes con `dtypes` (ten en cuenta que no es `dtype` como con los objetos `Series` e `Index` ya que cada columna tendrá su propio tipo de datos):

In [ ]:
df.dtypes

Podemos obtener los datos subyacentes con el atributo `values`. Tenga en cuenta que esto se ve muy similar a nuestra representación NumPy inicial:

In [ ]:
df.values

Podemos aislar las columnas con el atributo `columns`. Observe que las columnas son en realidad un objeto `Index` sólo que en un eje diferente (las columnas son el índice horizontal mientras que las filas son el índice vertical).

In [ ]:
df.columns

Estos son algunos de los atributos más utilizados:

|Atributo| Returns |
| --- | --- |
| `dtypes` | Los tipos de datos de cada columna |
| `shape` | Dimensiones del objeto `DataFrame` en una tupla de la forma `(número de filas, número de columnas)` |
| `index` | El objeto `Index` a lo largo de las filas del objeto `DataFrame` |
| `columns` | El nombre de las columnas (como objeto `Index`) |
| `values` | Los datos del objeto `DataFrame` |
| `empty` | Comprueba si el objeto `DataFrame` está vacío |

Se puede acceder al objeto `Index` a lo largo de las filas del marco de datos mediante el atributo `index` (igual que con los objetos `Series`):

In [ ]:
df.index

Al igual que con los objetos `Series` e `Index`, podemos obtener las dimensiones del marco de datos con el atributo `shape`. El resultado es de la forma `(n filas, n columnas)`. Nuestro marco de datos tiene 5 filas y 6 columnas:

In [ ]:
df.shape

Tenga en cuenta que también podemos realizar operaciones aritméticas en dataframes. Pandas sólo realizará la operación cuando el índice y la columna coincidan. Aquí, demostramos la suma. Dado que la suma con cadenas significa concatenación, `pandas` concatenó las columnas de cadena (`time`, `place`, `magType`, y `alert`) a través de dataframes. Las columnas numéricas (`mag` y `tsunami`) se sumaron:

In [ ]:
df + df

<hr>
<div>
    <a href="../ch_01/introduccion_al_data_analisis.ipynb">
        <button style="float: left;">&#8592; Capitulo 1</button>
    </a>
    <a href="./2-creando_dataframes.ipynb">
        <button style="float: right;">Siguiente Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>